In [ ]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.autograd import grad as torch_grad
import torch.nn.functional as F

import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA

import math

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/FYP/Stock Market Group/DataSet/AAPL/AAPL_data_with_indicators.csv")
df.head()

,Datetime,Open,High,Low,Close,Adj Close,Volume,SMA,wma,ema,...,strength_line,signal_line,fi,kline,dline,atr,aroon_up,aroon_down,roc,psar
0,2023-04-03 13:30:00,164.270004,165.559998,164.220001,165.496704,165.496704,3833951,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.339996,NaN,NaN,NaN,164.220001
1,2023-04-03 13:35:00,165.494995,165.849899,165.360001,165.725006,165.725006,1398850,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.279275,NaN,NaN,NaN,164.220001
2,2023-04-03 13:40:00,165.732300,165.732300,165.440002,165.449997,165.449997,1077282,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.208777,NaN,NaN,NaN,164.220001
3,2023-04-03 13:45:00,165.440002,165.869995,165.229996,165.869995,165.869995,1138968,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.168150,NaN,NaN,NaN,164.285197
4,2023-04-03 13:50:00,165.869995,165.889999,165.270004,165.300003,165.300003,1160507,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.128996,NaN,NaN,NaN,164.380285


# Data Pre-process

In [ ]:
def preprocess(df):
  df.index = pd.to_datetime(df['Datetime'])
  cols = list(df)[1:]
  df_for_training = df[cols].astype(float)
  df_for_training =df_for_training.dropna()
  labels_df = df_for_training["Close"]
  features_df = df_for_training.drop(columns=["Close"])

  return features_df,labels_df

# Sliding Window

In [ ]:
def sliding_window(x, y, window):
  x_ = []
  y_ = []
  y_gan = []
  for i in range(window, x.shape[0]-4):
    tmp_x = x[i - window: i, :]
    tmp_y = y[i:i+5].flatten()
    tmp_y_gan = y[i - window: i + 5]
    x_.append(tmp_x)
    y_.append(tmp_y)
    y_gan.append(tmp_y_gan)
  x_ = np.array(x_)
  y_ = np.array(y_)
  y_gan = np.array(y_gan)
  return x_, y_, y_gan

In [ ]:
#TODO: save scalers
def normalize(train_x,train_y):
  x_scaler = MinMaxScaler(feature_range = (0, 1))
  y_scaler = MinMaxScaler(feature_range = (0, 1))
  train_x = x_scaler.fit_transform(train_x)

  train_y = (train_y.values.reshape(-1, 1))

  return train_x,train_y


#Split datasets

In [ ]:
def split_dataset(features_df,labels_df,y_gan):
    train_x = features_df[:training_duration]
    train_y = labels_df[:training_duration]#.values.reshape(-1,1)
    train_y_gan = y_gan[:training_duration]#.values.reshape(-1,1)

    test_x = features_df[training_duration:training_duration+testing_duration]
    test_y = labels_df[training_duration:training_duration+testing_duration]#.values.reshape(-1,1)
    test_y_gan = y_gan[training_duration:training_duration+testing_duration]#.values.reshape(-1,1)

    print(f'trainX: {train_x.shape} trainY: {train_y.shape}')
    print(f'testX: {test_x.shape} testY: {test_y.shape}')

    train_x = torch.from_numpy(np.array(train_x)).float()
    train_y = torch.from_numpy(np.array(train_y)).float()
    train_y_gan = torch.from_numpy(np.array(train_y_gan)).float()
    test_x = torch.from_numpy(np.array(test_x)).float()
    test_y = torch.from_numpy(np.array(test_y)).float()
    test_y_gan = torch.from_numpy(np.array(test_y_gan)).float()


    return train_x,train_y,train_y_gan,test_x,test_y,test_y_gan

#Model Implementation

In [ ]:
class Generator(nn.Module):
  def __init__(self, input_size):
    super().__init__()

    # 3 GRU layers, input_size = features
    self.gru_1 = nn.GRU(input_size, 1024, batch_first=True)
    self.gru_2 = nn.GRU(1024, 512, batch_first = True)
    self.gru_3 = nn.GRU(512, 256, batch_first = True)
    self.gru_4 = nn.GRU(256, 128, batch_first = True)
    # 3 Dense Layers
    self.linear_1 = nn.Linear(256, 128)
    self.linear_2 = nn.Linear(128, 64)
    self.linear_3 = nn.Linear(64, 5)

    self.dropout = nn.Dropout(0.2)


  def forward(self, x):
    use_cuda = 1
    device = torch.device("cuda" if (torch.cuda.is_available() & use_cuda) else "cpu")
    h0 = torch.zeros(1, x.size(0), 1024).to(device) # initial hidden state for the 1st GRU Layer - (num of layers in the GRU, batch size, num of hidden units in the GRU)
    out_gru_1, _ = self.gru_1(x, h0)
    out_gru_1 = self.dropout(out_gru_1)

    h1 = torch.zeros(1, x.size(0), 512).to(device)
    out_gru_2, _ = self.gru_2(out_gru_1, h1)
    out_gru_2 = self.dropout(out_gru_2)

    h2 = torch.zeros(1, x.size(0), 256).to(device)
    out_gru_3, _ = self.gru_3(out_gru_2, h2)
    out_gru_3 = self.dropout(out_gru_3)

    h3 = torch.zeros(1, x.size(0), 128).to(device)
    out_gru_4, _ = self.gru_4(out_gru_3, h3)
    out_gru_4 = self.dropout(out_gru_4)


    out_dense_1 = self.linear_1(out_gru_3[:, -1, :])
    out_dense_2 = self.linear_2(out_dense_1)
    out_dense_3 = self.linear_3(out_dense_2)

    return out_dense_3,out_gru_4

In [ ]:
class Discriminator(nn.Module):
  def __init__(self):
    super().__init__()

    # 3 1D Conv layers
    self.conv1 = nn.Conv1d(sliding_window_size+5, 32, kernel_size = 5, stride = 1, padding = 'same')
    self.conv2 = nn.Conv1d(32, 64, kernel_size = 5, stride = 1, padding = 'same')
    self.conv3 = nn.Conv1d(64, 128, kernel_size = 5, stride = 1, padding = 'same')

    # 3 linear layers
    self.linear1 = nn.Linear(128, 220)
    self.linear2 = nn.Linear(220, 220)
    self.linear3 = nn.Linear(220, 5)

    self.leaky = nn.LeakyReLU(0.01)
    self.relu = nn.ReLU()
    self.tanh = nn.Tanh()

  def forward(self, x):
    conv1 = self.conv1(x)
    conv1 = self.leaky(conv1)
    conv2 = self.conv2(conv1)
    conv2 = self.leaky(conv2)
    conv3 = self.conv3(conv2)
    conv3 = self.leaky(conv3)

    flatten_x =  conv3.reshape(conv3.shape[0], conv3.shape[1])

    out_1 = self.linear1(flatten_x)
    out_1 = self.leaky(out_1)
    out_2 = self.linear2(out_1)
    out_2 = self.relu(out_2)
    out_3 = self.linear3(out_2)

    return out_3

In [ ]:
def grad_penalty_fnc(real_data, gen_data,D,cuda,gp_weight):
    batch_size = real_data.size()[0]
    t = torch.rand((batch_size, 1, 1), requires_grad=True)
    t = t.expand_as(real_data)

    if cuda:
        t = t.cuda()

    # mixed sample from real and fake; make approx of the 'true' gradient norm
    interpol = t * real_data.data + (1-t) * gen_data.data

    if cuda:
        interpol = interpol.cuda()
    prob_interpol = D(interpol)
    torch.autograd.set_detect_anomaly(True)
    gradients = torch_grad(outputs=prob_interpol, inputs=interpol,
                           grad_outputs=torch.ones(prob_interpol.size()).cuda() if cuda else torch.ones(
                               prob_interpol.size()), create_graph=True, retain_graph=True)[0]
    gradients = gradients.view(batch_size, -1)
    #grad_norm = torch.norm(gradients, dim=1).mean()
    #self.losses['gradient_norm'].append(grad_norm.item())

    # add epsilon for stability
    eps = 1e-10
    gradients_norm = torch.sqrt(torch.sum(gradients**2, dim=1, dtype=torch.double) + eps)
    #gradients = gradients.cpu()
    # comment: precision is lower than grad_norm (think that is double) and gradients_norm is float
    final = gp_weight * (torch.max(torch.zeros(1,dtype=torch.double).cuda() if cuda else torch.zeros(1,dtype=torch.double), gradients_norm.mean() - 1) ** 2), gradients_norm.mean().item()
    return final


#Train Model

In [ ]:
def train(train_x_slide,train_y_gan,scaled_x,sliding_window_size,device,cuda):
  batch_size = 128
  learning_rate = 0.000115
  num_epochs = 100
  critic_iterations = 5
  eraly_exit = 10
  treshold = 0.1
  count = 0

  trainDataloader = DataLoader(TensorDataset(train_x_slide, train_y_gan), batch_size = batch_size, shuffle = False)

  # Give number of features to the G
  modelG = Generator(scaled_x.shape[1]).to(device)
  modelD = Discriminator().to(device)

  #weight_decay-L2 penalty to the weights
  optimizerG = torch.optim.Adam(modelG.parameters(), lr = learning_rate, betas = (0.0, 0.9), weight_decay = 1e-3)
  optimizerD = torch.optim.Adam(modelD.parameters(), lr = learning_rate, betas = (0.0, 0.9), weight_decay = 1e-3)

  # optimizerG = torch.optim.RMSprop(modelG.parameters(), lr = learning_rate, weight_decay = 1e-3)
  # optimizerD = torch.optim.RMSprop(modelD.parameters(), lr = learning_rate, weight_decay = 1e-3)

  histG = np.zeros(num_epochs)
  histD = np.zeros(num_epochs)
  count = 0

  #k=0
  i=0
  for epoch in range(num_epochs):
    loss_G = []
    loss_D = []
    for (x, y) in trainDataloader:
      x = x.to(device)
      y = y.to(device)

      fake_data,_ = modelG(x)
#         print(y.shape,fake_data.shape)
#         print(y[:, :sliding_window_size, :].shape, fake_data.reshape(-1, 1, 1).shape)
      i=1
      fake_data = torch.cat([y[:, :sliding_window_size, :], fake_data.reshape(-1, 5, 1)], axis = 1)

      for _ in range(critic_iterations):
        critic_real = modelD(y)
        critic_fake = modelD(fake_data)
        grad_penalty, grad_norm_ = grad_penalty_fnc(y, fake_data,modelD,cuda,10)
        # Take probability mean of whole batch.
        lossD = -(torch.mean(critic_real) - torch.mean(critic_fake)) + grad_penalty

        modelD.zero_grad()
        lossD.backward(retain_graph = True)
        optimizerD.step()

      output_fake = modelD(fake_data)
      lossG = -torch.mean(output_fake)

      modelG.zero_grad() # zeroing the gradients
      lossG.backward() # computing the gradients
      optimizerG.step() # updating the parameters

      loss_D.append(lossD.item())
      loss_G.append(lossG.item())

    histG[epoch] = sum(loss_G)
    histD[epoch] = sum(loss_D)

    # Check if the loss exceeds the threshold
#     if sum(loss_D) > treshold:
#         count+=1
#         if count>= eraly_exit:
#             print(f'Early exit at epoch {epoch+1} due to loss exceeding the threshold.')
#             break
    print(f'[{epoch+1}/{num_epochs}] LossD: {sum(loss_D)} LossG:{sum(loss_G)}')
  return modelG

# Evaluate model

In [ ]:
def evaluateModel(modelG,train_x_slide,test_x_slide,train_y_slide,test_y_slide):
  modelG.eval()
  pred_y_train,_ = modelG(train_x_slide.to(device))
  pred_y_test,_ = modelG(test_x_slide.to(device))


  y_train_true =train_y_slide
  y_train_pred = pred_y_train.cpu().detach().numpy()

  y_test_true = test_y_slide
  y_test_pred = pred_y_test.cpu().detach().numpy()

  plt.figure(figsize=(12, 8))
  plt.plot(y_train_true, color = 'black', label = 'Acutal Price')
  plt.plot(y_train_pred, color = 'blue', label = 'Predict Price')
  plt.title('WGAN-GP prediction training dataset')
  plt.ylabel('BTC')
  plt.xlabel('5 min time periods')
  plt.legend(loc = 'upper right')

  MSE = mean_squared_error(y_train_true, y_train_pred)
  RMSE = math.sqrt(MSE)
  print(f'Training dataset RMSE:{RMSE}')


  plt.figure(figsize=(12, 8))
  plt.plot(y_test_true, color = 'black', label = 'Acutal Price')
  plt.plot(y_test_pred, color = 'blue', label = 'Predict Price')
  plt.title('WGAN-GP prediction testing dataset')
  plt.ylabel('BTC')
  plt.xlabel('5 min time periods')
  plt.legend(loc = 'upper right')

  MSE = mean_squared_error(y_test_true, y_test_pred)
  RMSE = math.sqrt(MSE)
  print(f'Testing dataset RMSE:{RMSE}')

#XGB input pre-process

In [ ]:
def xgb_input(train_input_features):
  # Create column names for features
  feature_names = [f"X_feature_{i}" for i in range(train_input_features.shape[1])]

  # Combine column names
  column_names = feature_names

  # Move the tensor to the CPU before converting to NumPy
  train_input_features_cpu = train_input_features.cpu().detach().numpy()

  # Create a DataFrame with the concatenated array and column names
  features_df = pd.DataFrame(train_input_features_cpu, columns=column_names)

  # Specify the path where you want to save the CSV file
  csv_path = "gan_output.csv"

  # Save the DataFrame to a CSV file
  features_df.to_csv(csv_path, index=False)

#Elliot input pre-process

In [ ]:
def nn_input(concatenated_original,all_predictions_original):
  # Get the last five values in each row of x_test1
  last_five_values_x_test1 = concatenated_original[:,-5:,0]
  # Create a DataFrame with the last five values and predicted values
  data = {'X_test1_Last_1': last_five_values_x_test1[:, 0],
          'X_test1_Last_2': last_five_values_x_test1[:, 1],
          'X_test1_Last_3': last_five_values_x_test1[:, 2],
          'X_test1_Last_4': last_five_values_x_test1[:, 3],
          'X_test1_Last_5': last_five_values_x_test1[:, 4],
          'Predicted_1': all_predictions_original[:, 0],
          'Predicted_2': all_predictions_original[:, 1],
          'Predicted_3': all_predictions_original[:, 2],
          'Predicted_4': all_predictions_original[:, 3],
          'Predicted_5': all_predictions_original[:, 4]}

  df = pd.DataFrame(data)

  # Specify the path where you want to save the CSV file
  csv_path = "output_file.csv"

  # Save the DataFrame to a CSV file
  df.to_csv(csv_path, index=False)

#Calculate

In [ ]:
#parameters
sliding_window_size = 10
# testing_duration = 2016
# training_duration = 2016*3

features_df,labels_df =preprocess(df)

# Dimentional Reduction
num_components = 31
pca = PCA(num_components)
features_df_pca = pca.fit_transform(features_df) # fit and reduce dimension

dataset_size= features_df.shape[0]
print("Full dataset size: ",dataset_size)
testing_duration = int(dataset_size*0.3)
training_duration = int(dataset_size*0.7)

scaled_x,scaled_y = normalize(features_df,labels_df) #without PCA

x_slide, y_slide, y_gan = sliding_window(scaled_x, scaled_y, sliding_window_size)
print(f'train_x: {x_slide.shape} train_y: {y_slide.shape} train_y_gan: {y_gan.shape}')

train_x_slide,train_y_slide,train_y_gan,test_x_slide,test_y_slide,test_y_gan = split_dataset(x_slide, y_slide, y_gan)
print(f'train_x: {train_x_slide.shape} train_y: {train_y_slide.shape} train_y_gan: {train_y_gan.shape}')
print(f'test_x: {test_x_slide.shape} test_y: {test_y_slide.shape} test_y_gan: {test_y_gan.shape}')

use_cuda = 1
cuda = torch.cuda.is_available()
device = torch.device("cuda" if (torch.cuda.is_available() & use_cuda) else "cpu")
print(device)

modelG = train(train_x_slide,train_y_gan,scaled_x,sliding_window_size,device,cuda)
evaluateModel(modelG,train_x_slide,test_x_slide,train_y_slide,test_y_slide)


#Get predictions to the full dataset
concatenated_array = np.concatenate((train_x_slide,test_x_slide), axis=0)
# Convert NumPy array to PyTorch tensor
concatenated_tensor = torch.tensor(concatenated_array, dtype=torch.float32).to(device)
# Forward pass through the model
all_predictions, gru_layer = modelG(concatenated_tensor)


train_input_features = gru_layer.reshape(gru_layer.shape[0], -1)
xgb_input(train_input_features)

labels_df = labels_df.to_numpy(dtype=float)
features_df = features_df.to_numpy(dtype=float)

original,_,_= sliding_window(features_df,labels_df,10)
original = original[:training_duration+testing_duration]

all_predictions_cpu = all_predictions.cpu().detach()
# Convert to NumPy array
all_predictions_numpy = all_predictions_cpu.numpy()
nn_input(original,all_predictions_numpy)



train_x: (12185, 10, 25) train_y: (12185, 5) train_y_gan: (12185, 15, 1)
trainX: (6048, 10, 25) trainY: (6048, 5)
testX: (2016, 10, 25) testY: (2016, 5)
train_x: torch.Size([6048, 10, 25]) train_y: torch.Size([6048, 5]) train_y_gan: torch.Size([6048, 15, 1])
test_x: torch.Size([2016, 10, 25]) test_y: torch.Size([2016, 5]) test_y_gan: torch.Size([2016, 15, 1])
